# Pytorch图像分类模型转ONNX

## 导入工具包

In [1]:
import torch
from torchvision import models

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 导入训练好的模型

In [2]:
import torch
import torchvision  # 确保 torchvision 在主脚本中可用
import torch.nn.functional as F 
from torch import nn
def get_net():
    model = torchvision.models.resnet50(pretrained = True) # 基于 ResNet50 预训练模型。
    model.avgpool = nn.AdaptiveAvgPool2d(1) # 修改了模型的平均池化层和全连接层。
    model.fc = nn.Linear(2048,59) # 适用于需要高效特征提取的分类任务。
    return model

checkpoint = torch.load('../checkpoints/best_model/0/model_best.pth.tar')
print(checkpoint.keys())
# 提取模型的参数
model = get_net()
model = torch.nn.DataParallel(model) # 使用 DataParallel 包装模型，以便在多GPU上并行训练。
device = torch.device('cuda' if torch.cuda.device_count() > 0 else 'cpu') 
    
# 打印检查点中 state_dict 的键值


model.load_state_dict(checkpoint['state_dict'])
torch.save(model, 'model_best.pth')

print("模型已保存为 model_best.pth 文件！")

dict_keys(['epoch', 'model_name', 'state_dict', 'best_precision1', 'optimizer', 'fold', 'valid_loss'])
模型已保存为 model_best.pth 文件！


In [3]:
model = torch.load('model_best.pth')
model = model.eval().to(device)

## 构造一个输入图像Tensor

In [4]:
x = torch.randn(1, 3, 256, 256).to(device)

## 输入Pytorch模型推理预测，获得59个类别的预测结果

In [5]:
output = model(x)

In [6]:
output.shape

torch.Size([1, 59])

## Pytorch模型转ONNX模型

In [12]:
x = torch.randn(1, 3, 256, 256).to(device)

# 解包为原始模型
if isinstance(model, torch.nn.DataParallel):
    model = model.module

# 然后再进行导出
torch.onnx.export(model,               # 解包后的模型
                  x,        # 输入 Tensor（需要符合模型的输入要求）
                  "resnet50-cls59.onnx",        # 导出文件名
                  export_params=True,  # 导出参数
                  opset_version=11,    # ONNX opset 版本
                  do_constant_folding=True,  # 是否执行常量折叠优化
                  input_names=['input'],     # 输入节点名称
                  output_names=['output'])   # 输出节点名称


## 验证onnx模型导出成功

In [14]:
import onnx

# 读取 ONNX 模型
onnx_model = onnx.load('resnet50-cls59.onnx')

# 检查模型格式是否正确
onnx.checker.check_model(onnx_model)

print('无报错，onnx模型载入成功')

无报错，onnx模型载入成功


## 以可读的形式打印计算图

In [15]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch-jit-export (
  %input[FLOAT, 1x3x256x256]
) initializers (
  %497[FLOAT, 64x3x7x7]
  %498[FLOAT, 64]
  %500[FLOAT, 64x64x1x1]
  %501[FLOAT, 64]
  %503[FLOAT, 64x64x3x3]
  %504[FLOAT, 64]
  %506[FLOAT, 256x64x1x1]
  %507[FLOAT, 256]
  %509[FLOAT, 256x64x1x1]
  %510[FLOAT, 256]
  %512[FLOAT, 64x256x1x1]
  %513[FLOAT, 64]
  %515[FLOAT, 64x64x3x3]
  %516[FLOAT, 64]
  %518[FLOAT, 256x64x1x1]
  %519[FLOAT, 256]
  %521[FLOAT, 64x256x1x1]
  %522[FLOAT, 64]
  %524[FLOAT, 64x64x3x3]
  %525[FLOAT, 64]
  %527[FLOAT, 256x64x1x1]
  %528[FLOAT, 256]
  %530[FLOAT, 128x256x1x1]
  %531[FLOAT, 128]
  %533[FLOAT, 128x128x3x3]
  %534[FLOAT, 128]
  %536[FLOAT, 512x128x1x1]
  %537[FLOAT, 512]
  %539[FLOAT, 512x256x1x1]
  %540[FLOAT, 512]
  %542[FLOAT, 128x512x1x1]
  %543[FLOAT, 128]
  %545[FLOAT, 128x128x3x3]
  %546[FLOAT, 128]
  %548[FLOAT, 512x128x1x1]
  %549[FLOAT, 512]
  %551[FLOAT, 128x512x1x1]
  %552[FLOAT, 128]
  %554[FLOAT, 128x128x3x3]
  %555[FLOAT, 128]
  %557[FLOAT, 512x128x1x1]
  %558

## 使用Netron可视化模型结构

Netron：https://netron.app